In [1]:
import pandas as pd

import warnings

warnings.filterwarnings("ignore")

In [47]:
df = pd.read_csv('/Users/Duy Tran/FootballAnalytics/Project 2 - Match Prediction Model/project2.csv')

In [48]:
df.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee
0,1.0,Fri,2018-08-17,20:15,Girona,0.7,0–0,0.0,Valladolid,10368.0,Estadi Municipal de Montilivi,Guillermo Cuadra
1,1.0,Fri,2018-08-17,22:15,Betis,0.9,0–3,2.5,Levante,46225.0,Estadio Benito Villamarín,Ignacio Iglesias
2,1.0,Sat,2018-08-18,18:15,Celta Vigo,0.6,1–1,0.6,Espanyol,16215.0,Estadio de Balaídos,Santiago Jaime
3,1.0,Sat,2018-08-18,20:15,Villarreal,1.6,1–2,0.7,Real Sociedad,16250.0,Estadio de la Cerámica,Mario Melero
4,1.0,Sat,2018-08-18,22:15,Barcelona,3.2,3–0,0.3,Alavés,52356.0,Camp Nou,José Sánchez


In [49]:
df.shape

(4560, 12)

In [51]:
#Display null values
df[df.isnull().any(axis=1)]

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee
1029,24.0,Tue,2020-03-10,20:00,Eibar,2.0,1–2,1.2,Real Sociedad,NaN,Estadio Municipal de Ipurúa,David Medié
1030,28.0,Thu,2020-06-11,22:00,Sevilla,1.7,2–0,0.5,Betis,NaN,Estadio Ramón Sánchez Pizjuán,Antonio Matéu Lahoz
1031,28.0,Fri,2020-06-12,19:30,Granada,2.3,2–1,0.6,Getafe,NaN,Estadio Nuevo Los Cármenes,David Medié
1032,28.0,Fri,2020-06-12,22:00,Valencia,1.0,1–1,1.4,Levante,NaN,Estadio de Mestalla,Alberola Rojas
1033,28.0,Sat,2020-06-13,14:00,Espanyol,2.5,2–0,0.1,Alavés,NaN,RCDE Stadium,Pablo González
...,...,...,...,...,...,...,...,...,...,...,...,...
4555,38.0,Sun,2024-05-19,16:00,Brentford,NaN,NaN,NaN,Newcastle Utd,NaN,Gtech Community Stadium,NaN
4556,38.0,Sun,2024-05-19,16:00,Chelsea,NaN,NaN,NaN,Bournemouth,NaN,Stamford Bridge,NaN
4557,38.0,Sun,2024-05-19,16:00,Crystal Palace,NaN,NaN,NaN,Aston Villa,NaN,Selhurst Park,NaN
4558,38.0,Sun,2024-05-19,16:00,Liverpool,NaN,NaN,NaN,Wolves,NaN,Anfield,NaN


In [52]:
# Drop rows where there are missing values in 'Score', 'xG', 'xG.1', and 'Referee'
df = df.dropna(subset=['Score', 'xG', 'xG.1', 'Referee'])

# We'll also choose to drop the 'Attendance' column if not used for prediction
# Drop 'Attendance' and 'Time' columns as they may not be essential for the initial model
df = df.drop(columns=['Attendance', 'Time'])

In [58]:
df.head()

,Wk,Day,Date,Home,xG,Score,xG.1,Away,Venue,Referee
0,1.0,Fri,2018-08-17,Girona,0.7,0–0,0.0,Valladolid,Estadi Municipal de Montilivi,Guillermo Cuadra
1,1.0,Fri,2018-08-17,Betis,0.9,0–3,2.5,Levante,Estadio Benito Villamarín,Ignacio Iglesias
2,1.0,Sat,2018-08-18,Celta Vigo,0.6,1–1,0.6,Espanyol,Estadio de Balaídos,Santiago Jaime
3,1.0,Sat,2018-08-18,Villarreal,1.6,1–2,0.7,Real Sociedad,Estadio de la Cerámica,Mario Melero
4,1.0,Sat,2018-08-18,Barcelona,3.2,3–0,0.3,Alavés,Camp Nou,José Sánchez


In [59]:
#Split score into home and away score
df[['home_goals', 'away_goals']] = df['Score'].str.split('–', expand=True).astype(int)

In [60]:
def determine_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'Home win'
    elif row['home_goals'] < row['away_goals']:
        return 'Away win'
    else:
        return 'Draw'

df['result'] = df.apply(determine_result, axis=1)

In [61]:
#Convert day into date time variable
df['Date'] = pd.to_datetime(df['Date'])

df['Day'] = df['Date'].dt.day_name()

In [62]:
#Add season start column
df['season_start'] = df['Date'].apply(lambda x: x.year -1 if x.month < 8 else x.year)

In [63]:
df.head()

,Wk,Day,Date,Home,xG,Score,xG.1,Away,Venue,Referee,home_goals,away_goals,result,season_start
0,1.0,Friday,2018-08-17,Girona,0.7,0–0,0.0,Valladolid,Estadi Municipal de Montilivi,Guillermo Cuadra,0,0,Draw,2018
1,1.0,Friday,2018-08-17,Betis,0.9,0–3,2.5,Levante,Estadio Benito Villamarín,Ignacio Iglesias,0,3,Away win,2018
2,1.0,Saturday,2018-08-18,Celta Vigo,0.6,1–1,0.6,Espanyol,Estadio de Balaídos,Santiago Jaime,1,1,Draw,2018
3,1.0,Saturday,2018-08-18,Villarreal,1.6,1–2,0.7,Real Sociedad,Estadio de la Cerámica,Mario Melero,1,2,Away win,2018
4,1.0,Saturday,2018-08-18,Barcelona,3.2,3–0,0.3,Alavés,Camp Nou,José Sánchez,3,0,Home win,2018


In [64]:
#Convert Day into boolean values so the model can read
df = pd.get_dummies(df, columns=['Day'])

In [65]:
#Reset index so the order is not messed up
df.reset_index(drop=True, inplace=True)

In [66]:
#Creating a temporary dataset to add average goals for home and away teams
for x in df.Home.unique():
    temp_df = df[(df['Home'] == x) | (df['Away'] == x)]
    temp_df = temp_df.sort_values(['Date'])
    
    #Assign correct goal values for each team
    temp_df['goal_value_to_calculate'] = temp_df.apply(lambda y: y['home_goals'] if y['Home'] == x else y['away_goals'], axis=1)
    
    #Roll into last 5 goals of a team to calculate the average
    temp_df['rolling_avg_goals'] = temp_df['goal_value_to_calculate'].rolling(window=5, closed='left', min_periods=1).mean()
    
    #Seperate rolling avg goals for home and away teams
    for index, row in temp_df.iterrows():
        if row['Home'] == x:
            df.at[index, 'home_rolling_avg_goals'] = row['rolling_avg_goals']
        else:
            df.at[index, 'away_rolling_avg_goals'] = row['rolling_avg_goals']


In [67]:
#Creating a temporary dataset to add xG(expected goals) for home and away teams

for x in df.Home.unique():
    temp_df = df[(df['Home'] == x) | (df['Away'] == x)]
    temp_df = temp_df.sort_values(['Date'])
    
    #Assign correct goal values for each team
    temp_df['xG_value_to_calculate'] = temp_df.apply(lambda y: y['xG'] if y['Home'] == x else y['xG.1'], axis=1)
    
    #Roll into last 5 goals of a team to calculate the average
    temp_df['rolling_avg_xG'] = temp_df['xG_value_to_calculate'].rolling(window=5, closed='left', min_periods=1).mean()
    
    #Seperate rolling avg goals for home and away teams
    for index, row in temp_df.iterrows():
        if row['Home'] == x:
            df.at[index, 'home_rolling_avg_xG'] = row['rolling_avg_xG']
        else:
            df.at[index, 'away_rolling_avg_xG'] = row['rolling_avg_xG']


In [68]:
df.head()

,Wk,Date,Home,xG,Score,xG.1,Away,Venue,Referee,home_goals,...,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday,home_rolling_avg_goals,away_rolling_avg_goals,home_rolling_avg_xG,away_rolling_avg_xG
0,1.0,2018-08-17,Girona,0.7,0–0,0.0,Valladolid,Estadi Municipal de Montilivi,Guillermo Cuadra,0,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
1,1.0,2018-08-17,Betis,0.9,0–3,2.5,Levante,Estadio Benito Villamarín,Ignacio Iglesias,0,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
2,1.0,2018-08-18,Celta Vigo,0.6,1–1,0.6,Espanyol,Estadio de Balaídos,Santiago Jaime,1,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
3,1.0,2018-08-18,Villarreal,1.6,1–2,0.7,Real Sociedad,Estadio de la Cerámica,Mario Melero,1,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
4,1.0,2018-08-18,Barcelona,3.2,3–0,0.3,Alavés,Camp Nou,José Sánchez,3,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN


In [69]:
#Drop null values 
df = df.dropna(subset=['home_rolling_avg_goals', 'away_rolling_avg_goals', 'home_rolling_avg_xG', 'away_rolling_avg_xG'])

In [70]:
#Looking at team Brighton stats
df[(df['Home'] == 'Brighton') | (df['Away'] == 'Brighton')] [['Date', 'Home', 'Away', 'xG', 'xG.1', 'home_rolling_avg_xG', 'away_rolling_avg_xG']]

,Date,Home,Away,xG,xG.1,home_rolling_avg_xG,away_rolling_avg_xG
398,2018-08-19,Brighton,Manchester Utd,1.7,1.4,0.300000,1.50
405,2018-08-25,Liverpool,Brighton,1.6,0.6,3.300000,1.00
412,2018-09-01,Brighton,Fulham,2.8,1.6,0.866667,1.20
429,2018-09-17,Southampton,Brighton,2.0,1.4,1.500000,1.35
437,2018-09-22,Brighton,Tottenham,0.8,1.9,1.360000,1.54
...,...,...,...,...,...,...,...
4385,2024-03-10,Brighton,Nott'ham Forest,0.4,0.7,1.640000,1.68
4401,2024-03-31,Liverpool,Brighton,2.8,0.5,2.720000,1.44
4409,2024-04-03,Brentford,Brighton,0.6,1.1,1.600000,1.30
4419,2024-04-06,Brighton,Arsenal,0.5,3.4,0.940000,1.56


In [71]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [72]:
df.columns

Index(['Wk', 'Date', 'Home', 'xG', 'Score', 'xG.1', 'Away', 'Venue', 'Referee',
       'home_goals', 'away_goals', 'result', 'season_start', 'Day_Friday',
       'Day_Monday', 'Day_Saturday', 'Day_Sunday', 'Day_Thursday',
       'Day_Tuesday', 'Day_Wednesday', 'home_rolling_avg_goals',
       'away_rolling_avg_goals', 'home_rolling_avg_xG', 'away_rolling_avg_xG'],
      dtype='object')

In [76]:
# Define the features and target variable
features = [column for column in df.drop(columns=['Date', 'xG', 'xG.1', 'Home', 'Away', 'Referee', 'Venue', 'Score', 'result', 'home_goals', 'away_goals', 'season_start']).columns]

# Split the data into train and test sets
train_data = df[df['season_start'] <= 2022]
test_data = df[df['season_start'] == 2023]

X_train = train_data[features]
y_train = train_data['result']
X_test = test_data[features]
y_test = test_data['result']

In [78]:
# Train a Random Forest model
clf = RandomForestClassifier(random_state=1)
clf.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [79]:
# Make predictions
predictions = clf.predict(X_test)

In [81]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

0.4469096671949287

In [82]:
# confusion matrix
confusion_matrix(y_test, predictions)

array([[ 76,  23,  85],
       [ 57,  19,  88],
       [ 61,  35, 187]], dtype=int64)

In [30]:
# Our baseline model has an accuracy of
df['result'].value_counts(normalize=True)

result
Home win    0.457744
Away win    0.293914
Draw        0.248342
Name: proportion, dtype: float64

In [83]:
# Let's first try to tune the hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, 20]
}

grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1),
             param_grid={'max_depth': [5, 10, 15, 20],
                         'n_estimators': [50, 100, 200]})

In [32]:
grid_search.best_params_

{'max_depth': 5, 'n_estimators': 50}

In [84]:
# Train a Random Forest model with the best hyperparameters
clf = RandomForestClassifier(random_state=1, n_estimators=50, max_depth=5)
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

0.4928684627575277

In [85]:
df = pd.get_dummies(df, columns=['Home', 'Away', 'Referee', 'Venue'])
df.head()

,Wk,Date,xG,Score,xG.1,home_goals,away_goals,result,season_start,Day_Friday,...,Venue_The American Express Community Stadium,Venue_The City Ground,Venue_The Hawthorns,Venue_The John Smith's Stadium,Venue_Tottenham Hotspur Stadium,Venue_Turf Moor,Venue_Vicarage Road Stadium,Venue_Villa Park,Venue_Vitality Stadium,Venue_Wembley Stadium
10,2.0,2018-08-24,1.4,2–0,0.6,2,0,Home win,2018,True,...,False,False,False,False,False,False,False,False,False,False
11,2.0,2018-08-24,1.7,2–2,1.6,2,2,Draw,2018,True,...,False,False,False,False,False,False,False,False,False,False
12,2.0,2018-08-25,0.6,0–0,0.9,0,0,Draw,2018,False,...,False,False,False,False,False,False,False,False,False,False
13,2.0,2018-08-25,0.9,1–0,1.5,1,0,Home win,2018,False,...,False,False,False,False,False,False,False,False,False,False
14,2.0,2018-08-25,0.4,0–1,1.0,0,1,Away win,2018,False,...,False,False,False,False,False,False,False,False,False,False


In [86]:
# Define the features and target variable
features = [column for column in df.drop(columns=['Date', 'xG', 'xG.1', 'Score', 'result', 'home_goals', 'away_goals', 'season_start']).columns]

# Split the data into train and test sets
train_data = df[df['season_start'] <= 2022]
test_data = df[df['season_start'] == 2023]

X_train = train_data[features]
y_train = train_data['result']
X_test = test_data[features]
y_test = test_data['result']

In [87]:
# Find the best hyperparameters for the Random Forest model
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

grid_search.best_params_

{'max_depth': 15, 'n_estimators': 200}

In [88]:
# Train a Random Forest model with the new features
clf = RandomForestClassifier(random_state=1, n_estimators=200, max_depth=15)
clf.fit(X_train, y_train)

# Make predictions
predictions = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
accuracy

0.5182250396196514